# Detecting Subtle Anomalies in Agricultural Datasets

Real-world agricultural imagery presents unique challenges for anomaly detection. Leaves in orchards and plantations are captured under **uncontrolled environmental conditions** — changing illumination, variable backgrounds, and differences in distance or orientation. In addition, **image sizes and resolutions vary widely**, adding further complexity to building consistent models.

In this notebook, we’ll explore these challenges using the **RoCoLe dataset**, which contains images of coffee leaves under both healthy and diseased conditions. Many of these samples appear *visually similar*, even across classes, making it difficult to spot small-scale or early-stage anomalies. At the end of the notebook we will explore a tiled dataset and compare that approach for anomaly detection in agriculture. 

---

## Objectives

1. **Explore and curate** the original dataset with [FiftyOne](https://voxel51.com/fiftyone/), identifying a critical subset of visually similar healthy vs. unhealthy samples.  
2. **Investigate anomaly detection** approaches when differences are subtle and localized.  
3. **Compare two data preparation strategies:**
   - **Masking the leaf region** — isolating the target area to suppress background noise.  
   - **Tiling the patches** — splitting images into small, consistent crops to amplify local anomalies.  
4. **Train and evaluate** a *PaDiM* model under both strategies, analyzing how masking and tiling affect anomaly localization and detection performance.

---

This walkthrough highlights how *data-centric techniques* — careful curation, spatial priors, and representation consistency — can help models detect small, context-sensitive anomalies in agricultural datasets.


In [ ]:
# %pip install --upgrade pip
# %pip install fiftyone, torch, torchvision, umap
# %pip install anomalib
# %pip install anomalib[vlm_clip]
# %pip install gdown
# %pip install pycocotools
# %pip install sam2

In [ ]:
import fiftyone as fo # base library and app
import fiftyone.brain as fob # ML methods
import fiftyone.zoo as foz # zoo datasets and models
from fiftyone import ViewField as F # helper for defining views
import fiftyone.utils.huggingface as fouh # Hugging Face integration
import fiftyone.types as fot

## Download the original dataset from my Google Drive Folder. 
### 📦 Importing the RoCoLe Dataset from Google Drive

We’ll begin by importing the **RoCoLe (Robusta Coffee Leaf)** dataset directly from Google Drive.  
This dataset contains images of coffee leaves under **healthy** and **diseased** conditions, captured in natural environments with varying lighting, backgrounds, and scales.

Using `gdown`, we’ll download the compressed dataset (`rocole_original.zip`) and extract it locally.  
This will make the image data available for exploration and analysis with FiftyOne in the next steps.


- [Original dataset](https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/c5yvn32dzg-2.zip)
- [Paper](https://www.sciencedirect.com/science/article/pii/S2352340919307693)
- [My Google Drive link](https://drive.google.com/file/d/1FnObgIu_G2sQwUa5nfGMZxCVbiyoVa8E/view?usp=drive_link) 


In [ ]:
import gdown

# Download the coffee dataset from Google Drive

url = "https://drive.google.com/uc?id=1FnObgIu_G2sQwUa5nfGMZxCVbiyoVa8E"  # original
gdown.download(url, output="rocole_original.zip", quiet=False)
!unzip rocole_original.zip


## 📂 Loading the RoCoLe Dataset into FiftyOne

Once the dataset is extracted, we’ll load it into FiftyOne to enable interactive exploration and visualization.

The dataset follows the **COCO format**, which stores both **bounding box detections** and **segmentation masks** for each leaf image.  
We’ll use `fo.Dataset.from_dir()` to create a persistent FiftyOne dataset named `coffee_rocole_coco`.

Key parameters:
- `dataset_dir`: the local path to the dataset folder.  
- `dataset_type`: specifies the annotation format (`COCODetectionDataset`).  
- `label_types`: includes both `"detections"` and `"segmentations"` fields.  
- `include_id=True`: ensures each annotation keeps its unique identifier.  

Once loaded, the dataset will be saved persistently for future sessions.


In [ ]:
dataset_name = "coffee_rocole"  # change if you like
dataset_dir  = "rocole-DatasetNinja"

# Create (or overwrite) the dataset from a standard directory tree
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.COCODetectionDataset,  # or another supported type
    name=dataset_name,
    label_types=["detections", "segmentations"],
    # If your segmentations are polygons, keep them as polylines (optional):
    # use_polylines=True,
    include_id=True, 
)

dataset.persistent = True  # keep it around between sessions

print(dataset)

In [ ]:
session = fo.launch_app(dataset, port=5151, auto=False)

## Generating Image Embeddings with ResNet50 and Visualizing Similarities

To better understand the relationships between images in the RoCoLe dataset, we’ll extract **feature embeddings** using a pretrained **ResNet50** model from the FiftyOne Model Zoo.

These embeddings capture high-level visual information about each image — color, texture, and structure — which will later help us identify clusters of **similar** or **anomalous** samples.

Steps performed in this cell:
1. **Load the ResNet50 model** trained on ImageNet.  
2. **Compute embeddings** for all images in the dataset and store them in a new field called `"resnet50_embeddings"`.  
3. Use **UMAP** (Uniform Manifold Approximation and Projection) to reduce the embedding dimensions for visualization.  
   The result, stored under the `brain_key` `"resnet50_vis"`, allows us to plot images in a 2D similarity space within the FiftyOne App.

This embedding visualization helps identify patterns such as clusters of healthy vs. unhealthy leaves and outliers that may represent subtle anomalies.


In [ ]:
model = foz.load_zoo_model(
    "resnet50-imagenet-torch"
)  # load the ResNet50 model from the zoo

# Compute embeddings for the dataset — this might take a while on a CPU
dataset.compute_embeddings(model=model, embeddings_field="resnet50_embeddings")

# Dimensionality reduction using UMAP on the embeddings
fob.compute_visualization(
    dataset,
    embeddings="resnet50_embeddings",
    method="umap",
    brain_key="resnet50_vis",
)

## Exploring Leaf Patches with FiftyOne Brain Visualization

To study **localized variations** within each leaf image, we can analyze the **individual patches** (detections) rather than the full images.  
This approach helps focus on the leaf areas themselves, reducing background influence from soil, branches, or lighting differences.

In this step:
- We use `fiftyone.brain.compute_visualization()` to generate a **2D embedding** of all detected patches.  
- The **`patches_field`** parameter (`"detections"`) tells FiftyOne to work at the patch level rather than full images.  
- UMAP (`method="umap"`) is applied again for dimensionality reduction, producing an interpretable **similarity map** of leaf patches.  
- The results are stored under the `brain_key` `"patches_viz"`, allowing us to visualize the patch-level embedding space in the FiftyOne App.

This helps us discover clusters of visually similar patches and potentially identify **subtle anomalies** that differ in color, texture, or structure.


In [ ]:
import fiftyone.brain as fob

results = fob.compute_visualization(
    dataset,
    patches_field="detections",  # or your segmentation field
    brain_key="patches_viz",
    num_dims=2,
    method="umap",
    verbose=True,
    seed=51,
)

## Computing Patch Similarity with CLIP Embeddings

Next, we’ll compute a **similarity index** between all leaf patches to enable semantic search and comparison.  
While the previous UMAP visualization helped us see global relationships, this step lets us **quantitatively measure how similar** different leaf regions are to one another.

Using `fiftyone.brain.compute_similarity()`:
- The **`patches_field`** (`"detections"`) specifies which regions to compare.  
- The **CLIP model** (`"clip-vit-base32-torch"`) encodes each patch into a semantic embedding space, capturing both visual and contextual features.  
- The resulting **similarity index** is stored under the `brain_key` `"gt_sim"`.  

This allows us to:
- Search for patches similar to a given region of interest.  
- Identify visually related anomalies or healthy patterns.  
- Explore subtle visual cues that may not be obvious in the full image context.

Overall, this similarity computation provides a foundation for **content-based retrieval** and **fine-grained anomaly discovery** in the RoCoLe dataset.


In [ ]:
# Index ground truth objects by similarity
fob.compute_similarity(
    dataset,
    patches_field="detections",  # your field containing patches
    model="clip-vit-base32-torch", # or another supported model
    brain_key="gt_sim",
)

## Preparing Similar Leaf Patches for Anomaly Detection

To train and evaluate an anomaly detection model effectively, we’ll first isolate a subset of **visually similar patches** — regions that look alike but belong to either **healthy** or **unhealthy** classes.  
This helps us focus on cases where anomalies are subtle and occur within visually homogeneous groups (e.g., similar textures, lighting, and orientation).

Steps in this cell:

1. **Convert detections into patch samples**  
   Using `dataset.to_patches("detections")`, we extract each annotated leaf patch as a standalone sample. This enables patch-level training and visualization.

2. **Select a query patch**  
   We pick one patch (`query_patch_id`) as the reference and retrieve its most visually similar counterparts using the previously computed similarity index (`"gt_sim"`).

3. **Create two specialized subsets:**
   - **`healthy_view`** → patches labeled as *healthy*.  
   - **`anormal_view`** → patches labeled as *non-healthy* (i.e., diseased or anomalous).  

This setup prepares the foundation for our **PaDiM anomaly detection experiments**, where both groups share visual similarity, but subtle differences reveal the presence of disease or damage.


In [ ]:
patches_view = dataset.to_patches("detections")
session.view = patches_view

# Use your specific patch ID
# query_patch_id = "68f2a0732aa8709f10a96803"
query_patch_id = patches_view.take(1).first().id 

# Sort patches by similarity to the chosen patch
similar_patches_view = patches_view.sort_by_similarity(
    query_patch_id, k=700, brain_key="gt_sim"
)

# --- Create the healthy (normal) view ---
healthy_view = similar_patches_view.match(
    F("detections.label") == "healthy"
)

# --- Create the anormal (non-healthy) view ---
anormal_view = similar_patches_view.match(
    F("detections.label") != "healthy"
)


## Exporting Healthy and Anomalous Patches for Training

With our subsets of visually similar **healthy** and **non-healthy** (anomalous) leaf patches prepared, we now export them as separate image directories.  
This structure is ideal for training anomaly detection models such as **PaDiM**, which require folder-based datasets with distinct “normal” and “anomalous” categories.

In this step:

1. **Define output directories** for both classes:
   - `normal_dir` → healthy leaf patches  
   - `anormal_dir` → diseased or damaged leaf patches  

2. **Export each subset** using FiftyOne’s `export()` method with `ImageDirectory` format.  
   This creates two independent, ready-to-train datasets compatible with most anomaly detection pipelines.

The resulting directory layout will look like:

```
rocole_patches/
│
├── normal/
│ ├── patch_0001.png
│ ├── patch_0002.png
│ └── ...
│
└── anormal/
├── patch_1001.png
├── patch_1002.png
└── ...
```

These exports provide the clean, structured data we’ll use for training and evaluating PaDiM under different preprocessing strategies.



In [ ]:
import os

base_dir = os.path.expanduser("rocole_patches")

# Option B (Databricks-friendly): uncomment this instead
# base_dir = "/dbfs/FileStore/rocole_patches"

normal_dir  = os.path.join(base_dir, "normal")
anormal_dir = os.path.join(base_dir, "anormal")

# Ensure directories exist
os.makedirs(normal_dir, exist_ok=True)
os.makedirs(anormal_dir, exist_ok=True)

# --- Export ---
healthy_view.export(
    export_dir=normal_dir,
    dataset_type=fo.types.ImageDirectory,
    overwrite=True
)

anormal_view.export(
    export_dir=anormal_dir,
    dataset_type=fo.types.ImageDirectory,
    overwrite=True
)

# Create the anomalib detection model for detecting healthy and unhealthy leaves.

## Training the PaDiM Anomaly Detection Model

With our **healthy (normal)** and **anomalous (non-healthy)** leaf patch datasets prepared, we can now train a **PaDiM** (Patch Distribution Modeling) anomaly detection model using the [Anomalib](https://github.com/openvinotoolkit/anomalib) framework.

PaDiM is well-suited for this task because it models the distribution of feature embeddings extracted from pretrained backbones, allowing it to detect subtle deviations in texture, color, or structure — exactly the kind of anomalies present in coffee leaf diseases.

### Steps in this section:

1. **Environment setup**
   - Clear GPU and memory caches for a clean training start.
   - Ensure `PYTORCH_CUDA_ALLOC_CONF` is configured for efficient memory handling.

2. **Data preparation**
   - Create a `Folder` datamodule pointing to our exported patch directories (`normal` and `anormal`).
   - Anomalib automatically resizes all inputs to **256×256**, ensuring consistent patch size.
   - Adjust `train_batch_size` and `num_workers` to balance speed and memory usage.

3. **Model configuration**
   - Use a lightweight **ResNet18** backbone for PaDiM.
   - Extract mid-level features from layers `["layer2", "layer3"]` (or just `"layer3"` for limited memory).
   - `n_features` controls the feature dimensionality and memory footprint.

4. **Training**
   - Instantiate the **Anomalib Engine**, leveraging GPU acceleration.
   - Call `engine.fit()` to begin model training on the RoCoLe patch dataset.

Once training completes, you’ll have a PaDiM model specialized in identifying **subtle leaf anomalies** under visually similar conditions — a core challenge in real-world agricultural inspection.


In [ ]:
# --- env hygiene (optional but helpful) ---
import os, gc, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache(); gc.collect()

# --- Anomalib: datamodule + model + trainer ---
from anomalib.data import Folder
from anomalib.engine import Engine
from anomalib.models.image.padim.lightning_model import Padim  # direct import avoids WinCLIP deps

# Datamodule applies a 256x256 resize internally
datamodule = Folder(
    name="rocole",
    root="rocole_patches",
    normal_dir="normal",
    abnormal_dir="anormal",
    train_batch_size=4,      # reduce until it fits (8 → 4 → 2 → 1)
    eval_batch_size=4,
    num_workers=4,
)
datamodule.setup("fit")

# Lightweight PaDiM (fits easier)
model = Padim(
    backbone="resnet18",
    layers=["layer2", "layer3"],    # or ["layer3"] if memory is tight
    n_features=50,                  # 32–100; lower uses less memory
    pre_trained=True,
)

engine = Engine(
    accelerator="gpu",
    devices=1,
)

engine.fit(model=model, datamodule=datamodule)
# (optional) run validation/test afterwards


## Running Inference on the RoCoLe Patch Dataset

After training, we’ll use the trained **PaDiM** model to perform inference on all patches in the dataset.  
This step generates **anomaly predictions** — identifying which patches are normal and which show potential defects or disease symptoms.

### Key steps in this cell:

1. **Define label mapping**
   - `label_map = {0: "normal", 1: "anormal"}`  
     This maps the binary model outputs to human-readable class labels.

2. **Specify prediction field**
   - `pred_field = "predictions"` will store the model output for each sample.

3. **Run inference**
   - Using `engine.predict()`, we feed the dataset through the trained model.
   - The `return_predictions=True` flag ensures that predictions are returned as a list of batch dictionaries, containing:
     - `pred_label` → model’s binary decision (0 = normal, 1 = anomalous)  
     - `pred_score` → anomaly confidence score  
     - optional spatial outputs such as anomaly maps or masks.

This inference step enables us to evaluate how well the trained PaDiM model distinguishes **subtle abnormalities** in visually similar leaf patches.

Model should be saved in ```/results/Padim/rocole/latest/weights/lightning/model.ckpt```


In [ ]:
label_map    = {0: "normal", 1: "anormal"}
pred_field   = "predictions"

# 4) Run prediction via datamodule (returns list of batch dicts)
pred_batches = engine.predict(
    model=model,
    datamodule=datamodule,
    return_predictions=True,
)

## Inspecting the Model’s Prediction Output

In [ ]:
print(type(pred_batches))
print(pred_batches)
# or check only the first element
print(type(pred_batches[0]))

## Re-importing the Exported Patches into FiftyOne

After running inference with Anomalib, we’ll re-import the exported **RoCoLe patch dataset** into FiftyOne.  
This allows us to visualize and analyze the predictions interactively alongside their corresponding patch images.

In this step:
1. **Load the directory tree** of patch images (`rocole_patches/`) into a new FiftyOne dataset named `"coffee_rocole_original_patches2"`.  
   - The dataset uses the `ImageClassificationDirectoryTree` type, which automatically assigns labels (`normal` or `anormal`) based on folder names.  
   - The label field is stored under `"detections"` for consistency with earlier steps.  

2. **Make the dataset persistent**, so it remains available between sessions for further visualization, evaluation, and annotation.

This re-imported dataset (`dataset2`) will serve as the base for attaching the **PaDiM model’s predictions**, visualizing anomaly scores, and comparing ground-truth vs. predicted labels within the FiftyOne App.


In [ ]:
import fiftyone as fo
import fiftyone.types as fot

#dataset = fo.load_dataset("coffee_rocole_patches")
dataset_name = "coffee_rocole_original_patches"  # change if you like
dataset_dir  = "rocole_patches"

# Create (or overwrite) the dataset from a standard directory tree
dataset2 = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fot.ImageClassificationDirectoryTree,
    name=dataset_name,
    label_field="detections",   # the field where labels will go
)

dataset2.persistent = True  # keep it around between sessions

print(dataset2)

In [ ]:
session = fo.launch_app(dataset2, port=5152, auto=False)

In [ ]:
model = foz.load_zoo_model(
    "clip-vit-base32-torch"
)  # load the CLIP model from the zoo

# Compute embeddings for the dataset
dataset2.compute_embeddings(
    model=model, embeddings_field="clip_embeddings", batch_size=64
)

# Dimensionality reduction using UMAP on the embeddings
fob.compute_visualization(
    dataset2, embeddings="clip_embeddings", method="umap", brain_key="clip_vis_2"
)

## Generating Spatial Heatmaps with the NVLabs C-RADIO Model

To further localize and visualize potential anomaly regions within each patch, we’ll use the **C-RADIO V3-H** model developed by NVIDIA Research (NVLabs).  
This model produces **spatial heatmaps** that highlight regions of high anomaly probability, serving as an interpretable bridge between raw pixel space and model decision space.

### Steps performed in this cell:

1. **Register the model source**  
   The C-RADIO implementation is hosted on GitHub, so we register its repository as a custom FiftyOne Model Zoo source.

2. **Load the pre-trained model**  
   Using `foz.load_zoo_model()`, we load the `"nv_labs/c-radio_v3-h"` model with spatial output enabled.  
   - `output_type="spatial"` ensures pixel-level anomaly scores.  
   - `apply_smoothing=True` and `smoothing_sigma=0.51` slightly blur the heatmap to reduce noise.  
   - `feature_format="NCHW"` aligns tensor shape with model expectations.

3. **Apply the model**  
   The model is applied to all samples in `dataset2`, storing results in a new field called `"radio_heatmap"`.  
   Each sample now includes a **spatial anomaly map** indicating regions where the model detects deviations from normal patterns.

These heatmaps are crucial for our next steps, where we’ll combine them with geometric priors (like center keypoints or bounding boxes) to guide region segmentation using SAM2 and refine anomaly localization.


In [ ]:
import fiftyone.zoo as foz

# Register the RADIO model source (run this once per environment)
foz.register_zoo_model_source(
    "https://github.com/harpreetsahota204/NVLabs_CRADIOV3",
)
# Load a RADIO model with spatial output
spatial_model = foz.load_zoo_model(
    "nv_labs/c-radio_v3-h",
    output_type="spatial",
    apply_smoothing=True,      # Optional: smooth the heatmap
    smoothing_sigma=0.51,      # Smoothing strength
    feature_format="NCHW"
)

# Apply the model to generate heatmaps
dataset2.apply_model(spatial_model, "radio_heatmap")

## Running PaDiM Inference and Writing Results to FiftyOne

This function defines a complete inference pipeline that integrates **Anomalib’s PaDiM model** with **FiftyOne datasets**.  
It handles model loading, prediction, tensor-to-array conversion, and structured field writing — all in a single, reusable function.

### Overview

The main function, `run_inference()`, performs the following steps:

1. **Model Resolution**
   - Dynamically imports the correct `Padim` class across multiple Anomalib versions.
   - Loads the trained model from a specified checkpoint (`.ckpt`).

2. **Per-sample Inference**
   - Iterates through all samples in a FiftyOne dataset or view (`sample_collection`).
   - Runs inference on each image using `engine.predict()`.
   - Collects both scalar and spatial predictions from Anomalib:
     - `pred_score`: anomaly confidence score.  
     - `pred_label`: binary anomaly flag (0 = normal, 1 = anomaly).  
     - `anomaly_map`: pixel-wise heatmap showing anomalous intensity.  
     - `pred_mask`: optional segmentation mask for localized defects.

3. **Automatic Field Writing**
   - Saves the following fields to each FiftyOne sample:
     - `pred_anomaly_score_<key>` → anomaly confidence (float).  
     - `pred_anomaly_<key>` → classification label (“normal” or “anomaly”).  
     - `pred_anomaly_map_<key>` → spatial heatmap (as `fo.Heatmap`).  
     - `pred_defect_mask_<key>` → segmentation mask (as `fo.Segmentation`).  

4. **Helper Utilities**
   - `_resolve_padim_cls()` ensures compatibility with all Anomalib versions.  
   - `_to_numpy2d()` converts tensors and nested dataclasses to clean 2D NumPy arrays.  
   - `_to_float()` safely converts torch scalars to Python floats.

### Example Usage
```python
from anomalib.engine import Engine
engine = Engine(accelerator="gpu", devices=1)

dataset2 = fo.load_dataset("coffee_rocole_original_patches2")
ckpt = "/home/paula/projects/databricks/results/Padim/rocole/latest/weights/lightning/model.ckpt"

run_inference(engine, dataset2, ckpt, key="padim", threshold=0.5)


In [ ]:
# --- requirements: anomalib, torch, fiftyone ---
# Assumes you already created `engine = Engine()` (or similar) elsewhere.

import importlib
import numpy as np
import fiftyone as fo

# ---------------------------
# Helpers
# ---------------------------

def _resolve_padim_cls():
    """
    Resolve the Padim class across multiple anomalib versions.
    """
    candidates = (
        "anomalib.models.image.padim.lightning_model.Padim",  # newer
        "anomalib.models.image.padim.Padim",                  # alt re-export
        "anomalib.models.padim.lightning_model.Padim",        # older
        "anomalib.models.padim.Padim",                        # older
        "anomalib.models.Padim",                              # very old
    )
    last_err = None
    for path in candidates:
        try:
            m, c = path.rsplit(".", 1)
            return getattr(importlib.import_module(m), c)
        except Exception as e:
            last_err = e
    raise ModuleNotFoundError(f"Could not import Padim from known paths. Last error: {last_err}")

def _as_numpy(x):
    """Try the common, non-recursive ways to get a numpy array."""
    if x is None:
        return None

    # Already numpy
    if isinstance(x, np.ndarray):
        return x

    # torch.Tensor -> numpy
    try:
        import torch  # noqa: F401
        if hasattr(x, "detach") and hasattr(x, "cpu") and hasattr(x, "numpy"):
            return x.detach().cpu().float().numpy()
    except Exception:
        pass

    # x.numpy() or x.to_numpy()
    for m in ("numpy", "to_numpy", "to_ndarray", "__array__"):
        try:
            meth = getattr(x, m, None)
            if callable(meth):
                arr = meth()
                if isinstance(arr, np.ndarray):
                    return arr
        except Exception:
            pass

    # Fallback: try np.asarray
    try:
        return np.asarray(x)
    except Exception:
        return None

def _to_numpy2d(x, *, dtype=None, normalize=False, _max_depth=5):
    """
    Convert various objects (torch.Tensor, PIL, Anomalib wrappers) into a 2D NumPy array.
    - For heatmaps: use dtype=np.float32 and normalize=True (scales to [0, 1])
    - For masks:    use dtype=np.int32   and normalize=False
    Returns None if conversion is not possible or not 2D.
    """
    if x is None:
        return None

    # First, try direct conversions (safe, non-recursive)
    arr = _as_numpy(x)

    # If still not numpy, carefully unwrap a few whitelisted attributes with cycle protection
    if not isinstance(arr, np.ndarray):
        visited = set()
        def _safe_peel(obj, depth=0):
            if obj is None or depth > _max_depth:
                return None
            oid = id(obj)
            if oid in visited:
                return None
            visited.add(oid)

            # Try converting this object directly
            arr_local = _as_numpy(obj)
            if isinstance(arr_local, np.ndarray):
                return arr_local

            # Only inspect a small, safe set of attributes that commonly hold the array/tensor
            for attr in ("anomaly_map", "heatmap", "mask", "data", "image", "value", "array", "tensor"):
                try:
                    if hasattr(obj, attr):
                        child = getattr(obj, attr)
                        if child is obj:
                            continue
                        out = _safe_peel(child, depth + 1)
                        if isinstance(out, np.ndarray):
                            return out
                except Exception:
                    # avoid properties that raise internally
                    continue
            return None

        arr = _safe_peel(x)

    if arr is None:
        return None

    # Squeeze stray dims
    arr = np.squeeze(arr)

    # If 3D, collapse a singleton or pick the first channel
    if arr.ndim == 3:
        if arr.shape[0] in (1, 3):  # (C, H, W) with C=1 or 3
            arr = arr[0]
        else:                       # (H, W, 1)
            arr = arr[..., 0]

    if arr.ndim != 2:
        return None

    if dtype is not None:
        arr = arr.astype(dtype, copy=False)

    if normalize:
        x_min, x_max = float(np.min(arr)), float(np.max(arr))
        rng = (x_max - x_min) or 1.0
        arr = (arr - x_min) / rng
        arr = arr.astype(np.float32, copy=False)

    return arr

def _to_float(x):
    return float(x.detach().cpu()) if hasattr(x, "detach") else float(x)

# ---------------------------
# Main function
# ---------------------------

def run_inference(engine, sample_collection, ckpt_path, key="padim", threshold=0.5):
    """
    Runs PaDiM inference via Anomalib Engine over a FiftyOne sample collection.

    Parameters
    ----------
    engine : anomalib.engine.Engine
        An initialized Anomalib Engine instance.
    sample_collection : fiftyone.core.collections.SampleCollection
        Any FiftyOne sample collection (Dataset or View).
    ckpt_path : str
        Path to a PaDiM Lightning checkpoint file (.ckpt).
    key : str, default "padim"
        Suffix used for field names written to each sample.
    threshold : float, default 0.5
        Score threshold at/above which samples are labeled "anomaly".

    Writes per-sample fields:
        - pred_anomaly_score_<key> : float
        - pred_anomaly_<key>       : fo.Classification("normal"/"anomaly")
        - pred_anomaly_map_<key>   : fo.Heatmap (float32 [0,1])  [if available]
        - pred_defect_mask_<key>   : fo.Segmentation (int32)     [if available]
    """
    # 1) Load the Anomalib model object from your checkpoint (do this once)
    Padim = _resolve_padim_cls()
    padim_model = Padim.load_from_checkpoint(ckpt_path)

    # 2) Iterate samples and call predict(), passing the model object each time
    for sample in sample_collection.iter_samples(autosave=True, progress=True):
        preds = engine.predict(
            model=padim_model,            # Anomalib model object (has .name)
            data_path=sample.filepath,    # Single image path is OK
            return_predictions=True,
        )
        actual = preds[0]                 # anomalib.data.dataclasses.torch.image.ImageBatch

        # Scalar score + label
        score = _to_float(actual.pred_score)
        label = "anomaly" if score >= threshold else "normal"

        # Write simple fields
        sample[f"pred_anomaly_score_{key}"] = score
        sample[f"pred_anomaly_{key}"] = fo.Classification(label=label)

        # Heatmap (float32 in [0,1])
        amap_np = _to_numpy2d(getattr(actual, "anomaly_map", None),
                              dtype=np.float32, normalize=True)
        if amap_np is not None:
            sample[f"pred_anomaly_map_{key}"] = fo.Heatmap(map=amap_np)

        # Segmentation mask (int32)
        pmask_np = _to_numpy2d(getattr(actual, "pred_mask", None),
                               dtype=np.int32, normalize=False)
        if pmask_np is not None:
            sample[f"pred_defect_mask_{key}"] = fo.Segmentation(mask=pmask_np)

# ---------------------------
# Example usage
# ---------------------------
# from anomalib.engine import Engine
# engine = Engine()  # configure as needed
# dataset2 = fo.load_dataset("your_dataset_name")  # or your existing collection
# ckpt = "/home/paula/projects/databricks/results/Padim/rocole/latest/weights/lightning/model.ckpt"
# run_inference(engine, dataset2, ckpt, key="padim", threshold=0.5)


## Inspecting PaDiM Predictions on a Single Sample

Before running batch-level evaluation, it’s useful to explore the output of the **PaDiM model** on an individual patch.  
This helps verify the prediction structure, inspect raw outputs, and understand which attributes are available for downstream visualization.

### What this cell does:

1. **Select a test image**  
   We take the first image from `dataset2` and load it with Pillow (`PIL.Image.open()`).

2. **Run inference**  
   Using `engine.predict()`, we pass the image through the trained PaDiM model.  
   Setting `return_predictions=True` returns a detailed prediction object rather than writing directly to disk.

3. **Inspect the returned data**
   - Print the overall type of the `preds` object and its first element.  
   - Explore its attributes and internal fields (e.g., `image_path`, `pred_score`, `pred_label`, `anomaly_map`, `pred_mask`).  
   - Convert tensor-based fields to native Python types for readability.

4. **Understand the model outputs**
   - `pred_score`: Anomaly confidence score — higher values indicate greater deviation from normality.  
   - `pred_label`: Binary classification (0 = *normal*, 1 = *anomalous*).  
   - `anomaly_map`: Pixel-wise anomaly intensity (if available).  
   - `pred_mask`: Segmentation mask of anomalous regions (if produced by the model).

This inspection helps confirm that the trained PaDiM model is returning the expected outputs and provides a foundation for visualizing heatmaps and integrating predictions into FiftyOne.


In [ ]:
from PIL import Image
## get the first sample from the test split
test_image = Image.open(dataset2.first().filepath)
Padim = _resolve_padim_cls()
padim_model = Padim.load_from_checkpoint("results/Padim/rocole/latest/weights/lightning/model.ckpt")


preds = engine.predict(
    model=padim_model,                      # optional if using ckpt/config  # "best", "last", or a path
    data_path=dataset2.first().filepath,   # single file is OK
    return_predictions=True
)

print(preds)
# 1. See type
print(type(preds))

# 2. If it's a list, check the first element
print(type(preds[0]))

# 3. Look at available attributes/keys
print(dir(preds[0]))          # if it's an object

# 4. Pretty-print contents
import pprint
pprint.pprint(preds[0])

sample = preds[0]  # anomalib.data.dataclasses.torch.image.ImageBatch

# --- quick glance at what's inside ---
print("available fields:", list(sample.__dataclass_fields__.keys()))

# --- the most useful attributes ---
print("path:", sample.image_path)          # str (path to the image)
print("pred_score:", sample.pred_score)     # float / tensor scalar
print("pred_label:", sample.pred_label)     # 0 (normal) or 1 (anomalous)

# Pixel-wise outputs (may be None depending on model/config)
print("pred_mask is None?", sample.pred_mask is None)   # segmentation-style mask
print("anomaly_map is None?", sample.anomaly_map is None)  # raw heatmap

# --- convert to Python scalars if tensors ---
to_float = (lambda x: float(x.detach().cpu()) if hasattr(x, "detach") else float(x))
print("pred_score (float):", to_float(sample.pred_score))
print("pred_label (int):", int(to_float(sample.pred_label)))


In [ ]:
run_inference(engine=engine, sample_collection=dataset2, ckpt_path= "results/Padim/rocole/latest/weights/lightning/model.ckpt", key="padim")

## Evaluating PaDiM Classification Performance  

With PaDiM predictions now in the dataset, we can **evaluate classification performance** against ground truth labels while aligning label names for consistency.

**Steps:**
1. **Setup:**  
   - `GT_FIELD = "detections"` → true labels (`normal`, `anormal`)  
   - `PRED_FIELD = "pred_anomaly_padim"` → model predictions  
   - `EVAL_KEY = "padim_eval"` → stores evaluation results  

2. **Normalize labels:**  
   ```python
   label_map = {"anomaly": "anormal"}
   mapped_view = dataset2.map_labels(PRED_FIELD, label_map)
   ```
3. **Run binary evaluation:**
   ```python
   results = mapped_view.evaluate_classifications(
    PRED_FIELD,
    gt_field=GT_FIELD,
    eval_key=EVAL_KEY,
    method="binary",
    classes=["normal", "anormal"],)
    results.print_report()
    ```
This computes metrics like accuracy, precision, recall, F1-score, and confusion matrix, viewable in the FiftyOne App or programmatically. It quantifies how well PaDiM distinguishes between healthy and anomalous leaf patches.


In [ ]:
GT_FIELD   = "detections"
PRED_FIELD = "pred_anomaly_padim"
EVAL_KEY   = "padim_eval"

# 1) Normalize prediction labels using map_labels
label_map = {"anomaly": "anormal"}
mapped_view = dataset2.map_labels(PRED_FIELD, label_map)

# 2) Evaluate as binary (positive = "anormal")
results = mapped_view.evaluate_classifications(
    PRED_FIELD,
    gt_field=GT_FIELD,
    eval_key=EVAL_KEY,
    method="binary",
    classes=["normal", "anormal"],
)

results.print_report()

In [ ]:
dataset2.persistent = True

## Leaf-Only Masking with SAM 2 (Center Seed Prompt)

To prevent **false anomalies outside the leaf**, we use SAM 2 to generate a **leaf mask** per image and later restrict anomaly maps to this region.  
Here we prompt SAM 2 with a **single keypoint at the image center**, asking it to segment the leaf around that point.

### What this cell does
1. **Create a center keypoint per sample** (`center_seed`)  
   - We add a `Keypoints` label with one point at the center of each image.
2. **Run SAM 2 from the FiftyOne Model Zoo**  
   - We apply the `"segment-anything-2-hiera-tiny-image-torch"` model, using the keypoint prompt to produce a leaf segmentation.  
   - The resulting mask(s) are stored in `sam2_mask`.

> **Important notes**
> - SAM 2’s keypoint prompts expect **pixel coordinates**, not normalized.  
>   If you created `points=[(0.5, 0.5)]`, convert to `(W//2, H//2)` per image.  
> - When prompting with keypoints, use `prompt_field="<your_field>_keypoints"` in `apply_model`.  
>   For the field `center_seed`, set `prompt_field="center_seed_keypoints"`.

After this step, you can **mask anomaly maps** or **ignore predictions** outside the leaf to reduce background-induced false positives.


In [ ]:
for sample in dataset2.iter_samples(progress=True, autosave=True):
    # Center point in relative coordinates
    center_x = 0.5
    center_y = 0.5

    # Create a Keypoints label with one Keypoint at the center
    sample["center_seed"] = fo.Keypoints(
        keypoints=[
            fo.Keypoint(label="center_seed", points=[(center_x, center_y)])
        ]
    )
    # No need to call sample.save() if autosave=True

In [ ]:
import fiftyone.zoo as foz

model = foz.load_zoo_model("segment-anything-2-hiera-tiny-image-torch")
dataset2.apply_model(model, label_field="sam2_mask", prompt_field="center_seed")

## Leaf Masking with SAM 2 (Five Keypoint Prompts)

To improve segmentation coverage and reduce **false anomalies at leaf edges**, we expand the SAM 2 prompt from one to **five keypoints** — one at the center and four around it (up, down, left, right).  
This helps SAM 2 better capture the entire leaf contour, especially when leaves are partially occluded or asymmetrical.

### What this cell does
1. **Generate five keypoints per image**  
   - Defines relative coordinates (0–1) for the center and four surrounding points (10% offset).  
   - Saves them in a new field `center_seed4` as a `Keypoints` label.  

2. **Run SAM 2 segmentation**  
   - Loads the `"segment-anything-2-hiera-tiny-image-torch"` model from the FiftyOne Zoo.  
   - Applies the model using the new `center_seed4` keypoints as prompts.  
   - Segmentation masks are stored in the `sam2_pred` field.  

> **Tip:**  
> When prompting SAM 2 with multiple keypoints, keep them evenly distributed across the target region for more stable mask generation.  

This step provides **denser and more robust leaf masks**, improving downstream anomaly detection by excluding irrelevant background regions.


In [ ]:
#import fiftyone as fo

for sample in dataset2.iter_samples(progress=True, autosave=True):
    # relative coordinates (0–1)
    cx, cy = 0.5, 0.5
    dy = 0.1  # 10% vertical offset
    dx = 0.1  # 10% horizontal offset (optional for symmetry)

    # center + four around center
    points = [
        (cx, cy),                # center
        (cx, cy - dy),           # up
        (cx, cy + dy),           # down
        (cx - dx, cy),           # left
        (cx + dx, cy),           # right
    ]

    sample["center_seed4"] = fo.Keypoints(
        keypoints=[fo.Keypoint(label="center_seed4", points=points)]
    )



In [ ]:
model = foz.load_zoo_model("segment-anything-2-hiera-tiny-image-torch")
dataset2.apply_model(model, label_field="sam2_pred", prompt_field="center_seed4")

## Post-Processing: AND SAM2 Mask with PaDiM Mask & Classify by Overlap

To reduce **background false positives** and only count anomalies that fall **within the leaf**, this step:
1. Builds a **full-image SAM2 leaf mask** from detections (`sam2_pred`).
2. Resizes the **PaDiM defect mask** (`pred_defect_mask_padim`) to image size.
3. Computes the **logical AND** between both masks to retain only defects **inside** the leaf area.
4. Converts the overlap ratio into a **binary classification** (`normal` vs. `anormal`).

### Configuration
- `SAM_FIELD = "sam2_pred"` — SAM2 masks (detections with per-detection masks).  
- `PADIM_FIELD = "pred_defect_mask_padim"` — PaDiM segmentation mask.  
- `OUT_AND_FIELD = "mask_and_sam2_padim"` — output segmentation (AND result).  
- `OUT_CLS_FIELD = "pred_by_overlap"` — output classification based on overlap.  
- `THRESH = 0.80` — minimum **coverage** of PaDiM mask by SAM2 (AND/PaDiM area) to call it **anormal**.  
- `OUT_DIR = "/tmp/masks_and"` — folder to export the AND masks as PNGs.

### What this cell does
1. **Reconstructs SAM2 full-image mask**  
   - Initializes an empty `H×W` boolean mask.  
   - For each detection in `sam2_pred`, grabs `det.get_mask()` (which is relative to the detection’s bounding box), **resizes it** to the box size, and **pastes** it back into the full-image coordinates.

2. **Normalizes PaDiM mask shape**  
   - Reads the PaDiM mask, binarizes it, and **resizes** (nearest neighbor) to `(H, W)` if needed to match image size.

3. **Combines masks & saves output**  
   - Computes `and_mask = sam_mask & padim_mask`.  
   - Saves `and_mask` via `fo.Segmentation(...).export_mask()` to `OUT_DIR`.  
   - Stores the segmentation in `mask_and_sam2_padim`.

4. **Computes coverage & classifies**  
   - `coverage = intersection(and_mask) / area(padim_mask)` (if PaDiM area > 0).  
   - If `coverage >= 0.80` → `anormal`, else `normal`.  
   - Writes a `Classification(label, confidence=coverage)` into `pred_by_overlap`.

### Why this helps
- Ensures anomalies are only counted **within the leaf** region predicted by SAM2.  
- Drops spurious detections on **background** or **borders**, improving precision.

### Notes & caveats
- Use **nearest-neighbor** resizing for binary masks (already set via `order=0`) to avoid soft edges.  
- If SAM2 misses parts of the leaf, coverage may be underestimated (potential false negatives).  
- If PaDiM area is tiny/noisy, a high threshold (0.80) can be strict; consider tuning per dataset.  
- The SAM2 detection mask is in **box-local coordinates**; this code correctly pastes it back using the detection’s bounding box.

### Next steps (optional)
- Evaluate `pred_by_overlap` against ground truth (e.g., `detections`) with:
  ```python
  ds.evaluate_classifications(
      "pred_by_overlap",
      gt_field="detections",
      eval_key="overlap_eval",
      method="binary",
      classes=["normal", "anormal"],
  ).print_report()


In [ ]:
import os
import numpy as np
import fiftyone as fo
from skimage.transform import resize

# ==== CONFIG ====
ds = dataset2
SAM_FIELD   = "sam2_pred"
PADIM_FIELD = "pred_defect_mask_padim"
OUT_AND_FIELD = "mask_and_sam2_padim"
OUT_CLS_FIELD = "pred_by_overlap"
THRESH = 0.80
OUT_DIR = "/tmp/masks_and"

os.makedirs(OUT_DIR, exist_ok=True)

ds.compute_metadata()

for s in ds.iter_samples(progress=True, autosave=True):
    sam_dets = s[SAM_FIELD] if SAM_FIELD in s else None
    padim_seg = s[PADIM_FIELD] if PADIM_FIELD in s else None
    
    if sam_dets is None or padim_seg is None:
        continue
    
    if not sam_dets.detections:
        continue
    
    # Get image dimensions
    H, W = s.metadata.height, s.metadata.width
    
    # Create full-size SAM mask
    sam_mask = np.zeros((H, W), dtype=bool)
    for det in sam_dets.detections:
        det_mask = det.get_mask()
        if det_mask is not None:
            bbox = det.bounding_box
            x, y, w, h = bbox
            x0, y0 = int(x * W), int(y * H)
            x1, y1 = int((x + w) * W), int((y + h) * H)
            
            det_mask_resized = resize(
                det_mask.astype(np.uint8),
                (y1 - y0, x1 - x0),
                order=0,
                preserve_range=True,
                anti_aliasing=False
            ).astype(bool)
            sam_mask[y0:y1, x0:x1] |= det_mask_resized
    
    # Get PaDiM mask
    padim_mask = padim_seg.get_mask()
    if padim_mask is None:
        continue
    
    padim_mask = (padim_mask > 0).astype(bool)
    
    # **RESIZE PaDiM mask to match image dimensions**
    if padim_mask.shape != (H, W):
        padim_mask = resize(
            padim_mask.astype(np.uint8),
            (H, W),
            order=0,  # nearest neighbor for binary masks
            preserve_range=True,
            anti_aliasing=False
        ).astype(bool)
    
    # AND operation (now both masks are same size)
    and_mask = sam_mask & padim_mask
    
    # Save using export_mask()
    out_path = os.path.join(OUT_DIR, f"{s.id}_and.png")
    and_seg = fo.Segmentation(mask=and_mask.astype(np.uint8) * 255)
    and_seg.export_mask(out_path, update=True)
    s[OUT_AND_FIELD] = and_seg
    
    # Calculate overlap
    padim_area = np.count_nonzero(padim_mask)
    inter_area = np.count_nonzero(and_mask)
    cov = (inter_area / padim_area) if padim_area > 0 else 0.0
    
    # Classification
    label = "anormal" if cov >= THRESH else "normal"
    s[OUT_CLS_FIELD] = fo.Classification(label=label, confidence=float(cov))

In [ ]:
dataset2.reload()

In [ ]:
# 2) Evaluate as binary (positive = "anormal")
results2 = mapped_view.evaluate_classifications(
    OUT_CLS_FIELD,
    gt_field="detections",
    eval_key="sam2_padim_eval",
    method="binary",
    classes=["normal", "anormal"],
)

results2.print_report()

In [ ]:
dataset2.persistent = True

## Generating Tiled Image Datasets with the FiftyOne Tile Plugin

This section demonstrates how to use the **`fiftyone-tile` plugin** by [@mmoollllee](https://github.com/mmoollllee/fiftyone-tile) to split large images into smaller, fixed-size tiles.  
This approach is particularly helpful for training, visual inspection, and patch-level anomaly detection.

### 1. Install and verify the plugin
We first download the plugin directly from GitHub and confirm it has been correctly loaded into FiftyOne.

```python
!fiftyone plugins download https://github.com/mmoollllee/fiftyone-tile/

import fiftyone.operators as foo

# List all available operators
operators = foo.list_operators()
for op in operators:
    print(op.uri)

# Verify that the tile operator is available
exists = foo.operator_exists("@mmoollllee/tile/make_tiles")
print(f"Operator exists: {exists}")

# Check for any plugin loading errors
from fiftyone.operators.registry import OperatorRegistry
registry = OperatorRegistry()
errors = registry.list_errors()
if errors:
    print("Plugin errors:")
    for error in errors:
        print(error)
```

Note: If the operator does not appear, restart your kernel or run fiftyone plugins list to confirm installation.


In [ ]:
!fiftyone plugins download https://github.com/mmoollllee/fiftyone-tile/
!fiftyone plugins requirements @mmoollllee/tile --install

In [ ]:
import fiftyone.operators as foo

# List all available operators
try:
    operators = foo.list_operators()
    print(f"Found {len(operators)} operators")
    for op in operators:
        print(op.uri)
except Exception as e:
    print(f"Error listing operators: {e}")

# Check specifically for your operator
try:
    exists = foo.operator_exists("@mmoollllee/tile/make_tiles")
    print(f"Operator exists: {exists}")
except Exception as e:
    print(f"Error checking operator: {e}")

# Check for any plugin loading errors
from fiftyone.operators.registry import OperatorRegistry
registry = OperatorRegistry()
errors = registry.list_errors()
if errors:
    print("Plugin errors:")
    for error in errors:
        print(error)
else:
    print("No plugin loading errors found")

### 2. Generate tiles from your dataset

Once the plugin is verified, we can call the @mmoollllee/tile/make_tiles operator to create a tiled dataset from our images.

In [ ]:
import fiftyone as fo
import fiftyone.operators as foo

# Get the operator using its URI
make_tiles = foo.get_operator("@mmoollllee/tile/make_tiles")

# Execute the operator using the __call__ method
make_tiles(
    dataset2,
    output_dir="tiled_images",
    destination="tiled_dataset",
    labels_field=None,  # Don't transfer any labels
    save_empty=True,    # Keep all tiles
    resize=1200,
    tile_size=256,
    padding=20,
    log_level=2
)

In [ ]:
# Load the tiled dataset (the destination you specified)
tiled_dataset = fo.load_dataset("tiled_dataset")

# Launch the FiftyOne App to visualize the tiles
session = fo.launch_app(tiled_dataset, port = 5152, auto=False)

# Let's test anomaly detection on a tiled dataset
 
We will use part of a tiled dataset of coffee leaves call [JMuBEN](https://www.kaggle.com/datasets/noamaanabdulazeem/jmuben-coffee-dataset). Just Healthy and Miner files as Nomarl and Anormal data, and we will use Anomalb and PaDiM model for detecting abnormal samples, and evaluate the classification task using Fiftyone. 

## Download the tiled dataset from my Google Drive Folder. 
### 📦 Importing the RoCoLe Dataset from Google Drive

We’ll begin by importing partial **JMuBEN** dataset directly from Google Drive.  
This dataset contains images of coffee leaves under **healthy** and **diseased** conditions, captured in natural environments with varying lighting, backgrounds, and scales.

Using `gdown`, we’ll download the compressed dataset (`rocole_original.zip`) and extract it locally.  
This will make the image data available for exploration and analysis with FiftyOne in the next steps.


- [Kaggle dataset](https://www.kaggle.com/datasets/noamaanabdulazeem/jmuben-coffee-dataset)
- [Original Dataset](https://data.mendeley.com/datasets/tgv3zb82nd/1)
- [Google Drive link](https://drive.google.com/file/d/12qZHphtkr4yJa7dWo0ob6Nl-53G_nIir/view?usp=sharing) 


In [ ]:
import fiftyone as fo

In [ ]:
import gdown

# Download the coffee dataset from Google Drive

url = "https://drive.google.com/uc?id=12qZHphtkr4yJa7dWo0ob6Nl-53G_nIir"  # original
gdown.download(url, output="partial_jmuben.zip", quiet=False)
!unzip partial_jmuben.zip


## 📂 Loading the JMuBEN Dataset into FiftyOne

Once the dataset is extracted, we’ll load it into FiftyOne to enable interactive exploration and visualization.

The dataset follows the **COCO format**, which stores both **bounding box detections** and **segmentation masks** for each leaf image.  
We’ll use `fo.Dataset.from_dir()` to create a persistent FiftyOne dataset named `coffee_rocole_coco`.

Key parameters:
- `dataset_dir`: the local path to the dataset folder.  
- `dataset_type`: specifies the annotation format (`ImageClassificationDirectoryTree`).  


Once loaded, the dataset will be saved persistently for future sessions.


In [ ]:
dataset_name = "partial_jmuben"  # change if you like
dataset_dir  = "coffee_leaves" # I have selected just two Healthy and Miner Folders from Kaggle page

# Create (or overwrite) the dataset from a standard directory tree
dataset3 = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fot.ImageClassificationDirectoryTree,
    name=dataset_name,
    label_field="ground_truth",   # the field where labels will go
)

dataset3.persistent = True  # keep it around between sessions

### ```create_index()```

This method creates a databesa index on specific fields to enable efficient sorting, merging and querying operations. We will apply that to the ```ground_truth.labels``` and later on to the predictions to the anomaly detection model. 

In [ ]:
dataset3.create_index("ground_truth.label")
dataset3.reload()

print(dataset3)

In [ ]:
session = fo.launch_app(dataset3, port=5153, auto=False)

### Explore your dataset with the Dashboard Plugin

Find the Dashboard Panel in FiftyOne and explore the distribution of your dataset and number of labels per category. 

In [ ]:
!fiftyone plugins download \
    https://github.com/voxel51/fiftyone-plugins \
    --plugin-names @voxel51/dashboard

In [ ]:
# --- env hygiene (optional but helpful) ---
import os, gc, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache(); gc.collect()

# --- Anomalib: datamodule + model + trainer ---
from anomalib.data import Folder
from anomalib.engine import Engine
from anomalib.models.image.padim.lightning_model import Padim  # direct import avoids WinCLIP deps

# Datamodule applies a 256x256 resize internally
datamodule = Folder(
    name="coffee_leaves",
    root="coffee_leaves",
    normal_dir="Healthy",
    abnormal_dir="Miner",
    train_batch_size=4,             # adjust for your GPU
    eval_batch_size=4,
    num_workers=4,
)
datamodule.setup("fit")

# Lightweight PaDiM (fits easier)
model = Padim(
    backbone="resnet18",
    layers=["layer2", "layer3"],    # or ["layer3"] if memory is tight
    n_features=50,                  # 32–100; lower uses less memory
    pre_trained=True,
)

engine = Engine(
    accelerator="gpu", #"cpu" is also an option Padin is CPU trainable
    devices=1,
)

engine.fit(model=model, datamodule=datamodule)
# (optional) run validation/test afterwards
# datamodule.setup("test"); engine.test(model=model, datamodule=datamodule)

In [ ]:

run_inference(engine=engine, sample_collection=dataset3, ckpt_path= "results/Padim/coffee_leaves/latest/weights/lightning/model.ckpt", key="padim")

In [ ]:
dataset3.persistent = True

In [ ]:
dataset3.create_index("pred_anomaly_padim.label")
dataset3.reload()

In [ ]:
GT_FIELD   = "ground_truth"
PRED_FIELD = "pred_anomaly_padim"
EVAL_KEY   = "padim_eval"

# 1) Normalize prediction labels using map_labels
label_map = {"Healthy": "normal", "Miner": "anormal"}
mapped_view = dataset3.map_labels(PRED_FIELD, label_map)

# 2) Evaluate as binary (positive = "anormal")
results = mapped_view.evaluate_classifications(
    PRED_FIELD,
    gt_field=GT_FIELD,
    eval_key=EVAL_KEY,
    method="binary",
    classes=["normal", "anormal"],
)

results.print_report()

In [1]:
import fiftyone as fo

dataset =fo.load_dataset("coffee_rocole")
dataset2 = fo.load_dataset("coffee_rocole_original_patches")
tiled_dataset = fo.load_dataset("tiled_dataset")

/Users/paularamos/Documents/GitHub/awesome-fiftyone/coffe_workshop_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You are running the oldest supported major version of MongoDB. Please refer to https://deprecation.voxel51.com for deprecation notices. You can suppress this exception by setting your `database_validation` config parameter to `False`. See https://docs.voxel51.com/user_guide/config.html#configuring-a-mongodb-connection for more information


In [ ]:
from fiftyone.utils.huggingface import push_to_hub

push_to_hub(tiled_dataset, "tiled_dataset")

Directory '/var/folders/6y/g2mslh_s7fz7qtj9vrxntqtm0000gn/T/tmp77b26skg' already exists; export will be merged with existing files
Exporting samples...
 100% |██████████████████| 9840/9840 [3.3s elapsed, 0s remaining, 3.1K docs/s]        


Uploading media files:   0%|          | 0/1 [00:00<?, ?it/s]It seems you are trying to upload a large folder at once. This might take some time and then fail if the folder is too large. For such cases, it is recommended to upload in smaller batches or to use `HfApi().upload_large_folder(...)`/`hf upload-large-folder` instead. For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/upload#upload-a-large-folder.


{"timestamp":"2025-10-23T14:26:28.927128Z","level":"WARN","fields":{"message":"Status Code: 429. Retrying...","request_id":""},"filename":"/Users/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":236}
{"timestamp":"2025-10-23T14:26:28.928456Z","level":"WARN","fields":{"message":"Status Code: 429. Retrying...","request_id":""},"filename":"/Users/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":236}
{"timestamp":"2025-10-23T14:26:28.933986Z","level":"WARN","fields":{"message":"Status Code: 429. Retrying...","request_id":""},"filename":"/Users/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":236}
{"timestamp":"2025-10-23T14:26:28.934067Z","level":"WARN","fields":{"message":"Status Code: 429. Retrying...","request_id":""},"filename":"/Users/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":236}
{"timestamp":"2025-10-23T14:26:28.936016Z","level":"WARN","fields":{"message":"Status Code: 429.

Uploading media files: 100%|██████████| 1/1 [03:38<00:00, 218.94s/it]


{"t":{"$date":"2025-10-23T14:20:50.682Z"},"s":"I",  "c":"CONTROL",  "id":20697,   "ctx":"-","msg":"Renamed existing log file","attr":{"oldLogPath":"/Users/paularamos/.fiftyone/var/lib/mongo/log/mongo.log","newLogPath":"/Users/paularamos/.fiftyone/var/lib/mongo/log/mongo.log.2025-10-23T14-20-50"}}


Subprocess ['/Users/paularamos/Documents/GitHub/awesome-fiftyone/coffe_workshop_env/lib/python3.10/site-packages/fiftyone/db/bin/mongod', '--dbpath', '/Users/paularamos/.fiftyone/var/lib/mongo', '--logpath', '/Users/paularamos/.fiftyone/var/lib/mongo/log/mongo.log', '--port', '0', '--nounixsocket'] exited with error -6:
